## Optimizing an ML Pipeline in Azure
### Overview
This project is part of the Udacity Azure ML Nanodegree. In this project, we build and optimize an Azure ML pipeline using the Python SDK and a provided Scikit-learn model. This model is then compared to an Azure AutoML run.

In [4]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-226827
Azure region: westeurope
Subscription id: 3e42d11f-d64d-4173-af9b-12ecaa1030b3
Resource group: aml-quickstarts-226827


In [5]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cluster_name = "compute-cluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

try:
    aml_compute = ComputeTarget(workspace=ws, name=cluster_name)
    print('An existing cluster will be used!')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2', max_nodes=4)
    aml_compute = ComputeTarget.create(ws, cluster_name, compute_config)
    print('An new cluster will be created now!')

aml_compute.wait_for_completion(show_output=True)

An existing cluster will be used!
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [6]:
from azureml.widgets import RunDetails
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--C': choice(0.1, 0.5, 1, 10, 20, 50),
        '--max_iter': choice(10, 50, 100, 200)
    }
)

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, delay_evaluation = 5, evaluation_interval = 1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='.',
                      command=['python', 'train.py'],
                      compute_target=aml_compute,
                      environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=20,
                                     max_concurrent_runs=4)

In [7]:
# Submit the hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(hyperdrive_config)
hyperdrive_run.wait_for_completion(show_output=True)
assert(hyperdrive_run.get_status() == "Completed")

RunId: HD_9b093f00-7cad-4cdb-a5c6-16982164638c
Web View: https://ml.azure.com/runs/HD_9b093f00-7cad-4cdb-a5c6-16982164638c?wsid=/subscriptions/3e42d11f-d64d-4173-af9b-12ecaa1030b3/resourcegroups/aml-quickstarts-226827/workspaces/quick-starts-ws-226827&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-02-26T21:21:53.757372][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-02-26T21:21:54.7578810Z][SCHEDULER][INFO]Scheduling job, id='HD_9b093f00-7cad-4cdb-a5c6-16982164638c_0' 
[2023-02-26T21:21:54.9147244Z][SCHEDULER][INFO]Scheduling job, id='HD_9b093f00-7cad-4cdb-a5c6-16982164638c_1' 
[2023-02-26T21:21:55.0212657Z][SCHEDULER][INFO]Scheduling job, id='HD_9b093f00-7cad-4cdb-a5c6-16982164638c_2' 
[2023-02-26T21:21:55.067825][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2023-02-26T21:21:55.2210326Z][SCHEDULER][INFO]Scheduling job, id='HD_9b093f00-7cad-4cdb-a5c6-1698216463

In [8]:
import joblib
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

print('Best Run Id: ', best_run.id)
print('\n Accuracy is', best_run_metrics['Accuracy'])
print('\n Regularization Strength is',best_run_metrics['Regularization Strength:'])
print('\n Max Iterations is',best_run_metrics['Max iterations:'])

best_run_metrics = best_run.get_metrics()

model = best_run.register_model(model_name='Model_HighAccuracy', model_path='./outputs/', 
                                properties={'Accuracy': best_run_metrics['Accuracy'],
                                            'Regularization Strength': best_run_metrics['Regularization Strength:'],
                                            'Max Iterations': best_run_metrics['Max iterations:']})

Best Run Id:  HD_9b093f00-7cad-4cdb-a5c6-16982164638c_0

 Accuracy is 0.9060451565914057

 Regularization Strength is 1.0

 Max Iterations is 100


In [7]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

dataset = TabularDatasetFactory.from_delimited_files([url])


In [8]:
from train import clean_data
from sklearn.model_selection import train_test_split

x, y = clean_data(dataset)

In [10]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
automl_config = AutoMLConfig(
    compute_target = aml_compute,
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=dataset,
    label_column_name='y',
    enable_onnx_compatible_models=True,
    n_cross_validations=2)

In [12]:
# Submit the automl run
automl_run = exp.submit(automl_config, show_output = False)
automl_run.wait_for_completion()

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_edcddd61-4408-40ae-a5ad-f8c7c484c294,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


{'runId': 'AutoML_edcddd61-4408-40ae-a5ad-f8c7c484c294',
 'target': 'compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2023-02-26T21:54:46.332167Z',
 'endTimeUtc': '2023-02-26T22:26:39.690637Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '2',
  'target': 'compute-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"3e42d11f-d64d-4173-af9b-12ecaa1030b3","resource_group":"aml-quickstarts-226827","workspace_name":"quick-starts-ws-226827","region":"westeurope","compute_target":"compute-cl

In [16]:
# Retrieve and save your best automl model.
best_run, fitted_model = automl_run.get_output()

print(best_run)
print(fitted_model)

# get_metrics()
# Returns the metrics
print("Best run metrics :",best_run.get_metrics())
# get_details()
# Returns a dictionary with the details for the run
print("Best run details :",best_run.get_details())

Run(Experiment: udacity-project,
Id: AutoML_edcddd61-4408-40ae-a5ad-f8c7c484c294_34,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=False, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=True, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('23', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False)), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bytree=1, eta=0.05, gamma=0, max_depth=6, max_leaves=0, n_estimators=200, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': '

In [22]:
best_run_metrics = best_run.get_metrics()

model = best_run.register_model(model_name='AutoML_HighAccuracy', model_path='./outputs/')

In [ ]:
# Cluster clean up

aml_compute.delete()